In [9]:
#!pip install rdflib
#% !pip install morph-kgc

In [10]:
# Hay un problemilla con la forma en la que se generan los enlaces con el morph, ya que por ejemplo 'http:/' se codifica como 'http%3A...',
# mientras no averiguemos como hacer para que genere la codificacion bien directamente, este codigo deberia corregir bien los enlaces

import re
from urllib.parse import unquote, quote

SAFE = ":/?#[]@!$&'()*+,;=%-._~"  # caracteres que NO queremos codificar en IRIs
ANGLE_IRI = re.compile(r"<([^>]*)>")

def fix_iri(iri: str) -> str:
    dec = unquote(iri)           # decodifica http%3A%2F%2F -> http://
    rec = quote(dec, safe=SAFE)  # vuelve a codificar sólo lo peligroso (espacios -> %20, etc.)
    return rec

with open("tripletas.nt", "r", encoding="utf-8") as src, open("tripletas_fixed.nt", "w", encoding="utf-8") as dst:
    for line in src:
        line = ANGLE_IRI.sub(lambda m: f"<{fix_iri(m.group(1))}>", line)
        dst.write(line)

print("OK -> tripletas_fixed.nt")

FileNotFoundError: [Errno 2] No such file or directory: 'tripletas.nt'

In [ ]:
# DESCOMENTAR SI SE TIENEN QUE GENERAR TODAVIA LAS TRIPLETAS

import morph_kgc
g = morph_kgc.materialize('config.ini')
g.serialize(destination='resultado.nt', format='nt')
print("Archivo resultado.nt generado")

# DESCOMENTAR SI SE HAN GENERADO YA LAS TRIPLETAS
# from rdflib import Graph
# g = Graph()
# g.parse("ontology.ttl", format="turtle")
# g.parse("tripletas_fixed.nt", format="nt")

MissingSectionHeaderError: File contains no section headers.
file: '<string>', line: 1
'config.ini'

In [ ]:
PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX owl:    <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#>
"""


print(f"Triples cargados: {len(g)}")

# Mostrar las primeras 10 tripletas
for i, (s, p, o) in enumerate(g):
    print(s, p, o)
    if i >= 25:
        break

Triples cargados: 6305
http://smartcity.linkeddata.es/lcc/resource/facility/park/Parque-Arroyo-de-los-Pinos http://www.w3.org/2003/01/geo/wgs84_pos#long -3.7241915440169024
http://smartcity.linkeddata.es/lcc/resource/facility/park/Parque-Azorin https://schema.org/name Parque Azorín
http://smartcity.linkeddata.es/lcc/resource/facility/educationalInstitution/Colegio-Mayor-Pio-XII https://schema.org/postalCode 28040
http://smartcity.linkeddata.es/lcc/resource/facility/educationalInstitution/Escuela-Tecnica-Superior-de-Ingenieria-de-Montes,-Forestal-y-del-Medio-Natural http://www.w3.org/2003/01/geo/wgs84_pos#long -3.724.507.841.206.360
http://smartcity.linkeddata.es/lcc/resource/facility/educationalInstitution/Escuela-Tecnica-Superior-de-Ingenieria-de-Montes,-Forestal-y-del-Medio-Natural http://smartcity.linkeddata.es/lcc/ontology#hasNearby http://smartcity.linkeddata.es/lcc/resource/subway/Metro:%20Vicente%20Aleixandre%20o%20Ciudad%20Universitaria(l%C3%ADnea%206)
http://smartcity.linkedda

In [ ]:
print(f"Triples cargados: {len(g)}")

PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX owl:    <http://www.w3.org/2002/07/owl#>
"""

def qrun(q, limit=50):
    q = PREFIXES + q
    res = g.query(q)
    print(f"--- {len(res)} filas ---")
    for i,row in enumerate(res):
        print([str(x) if x is not None else None for x in row])
        if i >= limit-1: break

Triples cargados: 6305


In [ ]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:EducationalInstitution   BIND("EducationalInstitution"    AS ?type) } UNION
  { ?x a schema:Library      BIND("PublicLibrary" AS ?type) } UNION
  { ?x a schema:ParkGarden   BIND("ParkGarden"    AS ?type) } UNION
  { ?x a sc:SportsCenter BIND("SportsCenter"  AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 4 filas ---
['EducationalInstitution', '72']
['ParkGarden', '128']
['PublicLibrary', '50']
['SportsCenter', '59']


In [ ]:
# 🔎 5 ejemplos de cada subclase (solo nombre)
# - Lista 5 nombres por subclase (si falta schema:name, usa el localname del IRI)

for subtype,label in [
    ("sc:EducationalInstitution","EducationalInstitutions"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {subtype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### EducationalInstitutions
--- 5 filas ---
['Universidad Complutense de Madrid. Centro Complutense para la Enseñanza del Español']
['Universidad Autónoma de Madrid. Facultad de Filosofía y Letras']
['Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación']
['Residencia Universitaria Nebrija Arturo Soria']
['Colegio Mayor Nuestra Señora de Guadalupe']

### PublicLibraries
--- 5 filas ---
['Biblioteca Pública Municipal David Gistau (Salamanca)']
['Biblioteca Pública Municipal Ángel González (Latina)']
['Biblioteca Pública Municipal Gabriel García Márquez (Usera)']
['Biblioteca Pública Municipal San Blas (San Blas-Canillejas)']
['Biblioteca Pública del Estado Manuel Alvar  (Salamanca)']

### ParksGardens
--- 5 filas ---
['Parque Hortaleza']
['Área Forestal de Tres Cantos']
['Parque Concejal Miguel Martín Vela']
['Jardín de Embajadores M-30']
['Parque Arroyo Fontarrón']

### SportsCenters
--- 5 filas ---
['Centro Deportivo Municipal Antiguo Canódromo']
['Centro Deportivo M

In [ ]:
# 🚇 Número de individuos de Metro, Bus y Tren (Cercanías)
# - Tabla con tipo de transporte y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a schema:SubwayStation        BIND("Subway"        AS ?type) } UNION
  { ?x a schema:BusStation           BIND("Bus"           AS ?type) } UNION
  { ?x a schema:TrainStation BIND("SuburbanTrain" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 3 filas ---
['Bus', '254']
['SuburbanTrain', '50']
['Subway', '263']


In [ ]:
# 🚏 5 ejemplos de Metro, Bus y Tren (solo nombre)
# - Muestra 5 nombres por tipo de transporte

for ttype,label in [
    ("schema:SubwayStation","Subway"),
    ("schema:BusStation","Bus"),
    ("schema:TrainStation","SuburbanTrain")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?t a {ttype} .
  OPTIONAL {{ ?t schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?t), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Subway
--- 5 filas ---
['Metro: Tribunal, Alonso Martínez, Bilbao']
['Metro: Ciudad de los Angeles']
['Metro: línea 1 (Atocha), línea 2 (Banco de España)']
['Metro: Embajadores (líneas 3 y 5), Atocha (línea 1)']
['Metro: Cuzco (línea 10)']

### Bus
--- 5 filas ---
['Bus%3A%2031%20%2C%2033%20%2C%2036%20%2C%2039%20%2C%2065%20%2C%20138']
['Bus%3A%203%2C%2040%20y%20147']
['Bus%3A%20%2076']
['Bus%3A%2023%20%2C%20123%20%2C%20130']
['Bus%3A%207%2C%209%2C%2070%2C%2072%20y%2087']

### SuburbanTrain
--- 5 filas ---
['Renfe: AtochaAlcalá de Henares (líneas C2 y C7) C1,C2,C3,C4,C5,C6,C7,C8 y C10']
['Renfe: Méndez Álvaro c1 c5 c10']
['Renfe: C1, C2 Y C7']
['Renfe: MirasierraPaco de Lucía (líneas C7, C8, C8a, C8b)']
['Renfe: Atocha C1,C2,C3,C4,C5,C6,C7,C8 y C10']


In [ ]:
# 🗺️ Número de individuos de Municipio, Distrito y Barrio
# - Tabla con tipo territorial y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a dbo:Municipality  BIND("Municipality"  AS ?type) } UNION
  { ?x a dbo:District      BIND("District"      AS ?type) } UNION
  { ?x a dbo:Neighbourhood BIND("Neighbourhood" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 3 filas ---
['District', '22']
['Municipality', '1']
['Neighbourhood', '118']


In [ ]:
# 🏷️ 5 ejemplos de Municipio, Distrito y Barrio (solo nombre)
# - Muestra 5 nombres por cada clase territorial

for ctype,label in [
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood"),
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {ctype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Municipality
--- 1 filas ---
['MADRID']

### District
--- 5 filas ---
['TETUAN']
['VILLAVERDE']
['ARGANZUELA']
['CENTRO']
['SAN BLAS-CANILLEJAS']

### Neighbourhood
--- 5 filas ---
['FUENTELARREINA']
['LOS ANGELES']
['ENTREVIAS']
['LOS ROSALES']
['VISTA ALEGRE']


In [ ]:
# 🔁 Comprobar jerarquía Municipio→Distrito→Barrio
# - Lista pares Municipio-Distrito y Distrito-Barrio (nombres) para verificar la estructura

print("### Neigbourhood → District")
qrun("""
SELECT ?municipality ?district WHERE {
  ?m a dbo:Neighbourhood ; schema:name ?municipality ; sc:locatedInDistrict ?d .
  OPTIONAL { ?d schema:name ?district }
}
ORDER BY ?municipality ?district
""", limit=20)

print("\n### District → Municipality")
qrun("""
SELECT ?district ?neighbourhood WHERE {
  ?d a dbo:District ; schema:name ?district ; sc:locatedInMunicipality ?n .
  OPTIONAL { ?n schema:name ?neighbourhood }
}
ORDER BY ?district ?neighbourhood
""", limit=20)


### Neigbourhood → District
--- 119 filas ---
['ABRANTES', 'CARABANCHEL']
['ACACIAS', 'ARGANZUELA']
['ADELFAS', 'RETIRO']
['ALAMEDA DE OSUNA', 'BARAJAS']
['ALMAGRO', 'CHAMBERI']
['ALMENARA', 'TETUAN']
['ALMENDRALES', 'USERA']
['ALUCHE', 'LATINA']
['AMBROZ', 'VICALVARO']
['APOSTOL SANTIAGO', 'HORTALEZA']
['ARAPILES', 'CHAMBERI']
['ARAVACA', 'MONCLOA-ARAVACA']
['ARCOS', 'SAN BLAS-CANILLEJAS']
['ARGUELLES', 'MONCLOA-ARAVACA']
['ATALAYA', 'CIUDAD LINEAL']
['BELLAS VISTAS', 'TETUAN']
['BERRUGUETE', 'TETUAN']
['BUENAVISTA', 'CARABANCHEL']
['BUTARQUE', 'VILLAVERDE']
['CANILLAS', 'HORTALEZA']

### District → Municipality
--- 22 filas ---
['ARGANZUELA', 'MADRID']
['BARAJAS', 'MADRID']
['CARABANCHEL', 'MADRID']
['CENTRO', 'MADRID']
['CHAMARTIN', 'MADRID']
['CHAMBERI', 'MADRID']
['CIUDAD LINEAL', 'MADRID']
['DISTRITO', 'MADRID']
['FUENCARRAL-EL PARDO', 'MADRID']
['HORTALEZA', 'MADRID']
['LATINA', 'MADRID']
['MONCLOA-ARAVACA', 'MADRID']
['MORATALAZ', 'MADRID']
['PUENTE DE VALLECAS', 'MADRID']
['RE

In [ ]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)



### Universities → Neighbourhood
--- 5 filas ---
['Universidad Complutense de Madrid. Centro Complutense para la Enseñanza del Español', 'MONCLOA-ARAVACA']
['Universidad Autónoma de Madrid. Facultad de Filosofía y Letras', 'FUENCARRAL-EL PARDO']
['Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación', 'DISTRITO']
['Residencia Universitaria Nebrija Arturo Soria', 'HORTALEZA']
['Colegio Mayor Nuestra Señora de Guadalupe', 'MONCLOA-ARAVACA']

### PublicLibraries → Neighbourhood
--- 5 filas ---
['Biblioteca Pública Municipal David Gistau (Salamanca)', 'SALAMANCA']
['Biblioteca Pública Municipal Ángel González (Latina)', 'LATINA']
['Biblioteca Pública Municipal Gabriel García Márquez (Usera)', 'USERA']
['Biblioteca Pública Municipal San Blas (San Blas-Canillejas)', 'SAN BLAS-CANILLEJAS']
['Biblioteca Pública del Estado Manuel Alvar  (Salamanca)', 'SALAMANCA']

### ParksGardens → Neighbourhood
--- 5 filas ---
['Parque Hortaleza', 'HORTALEZA']
['Área Forestal de Tres Cantos'

In [ ]:
# 🚈 Comprobar relaciones subclases de Facility → Metro/Bus/Tren
# - Para cada subclase: cuenta cuántas instalaciones tienen cada tipo de transporte cercano

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:

    print(f"\n### {label} con transporte cercano (conteos por tipo)")
    qrun(f"""
SELECT ?transportType (COUNT(DISTINCT ?f) AS ?n) WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?transportType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?transportType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?transportType)
  }}
}}
GROUP BY ?transportType
ORDER BY DESC(?n)
""", limit=10)

    print(f"### {label} → ejemplos (nombre facility, tipo transporte, nombre transporte)")
    qrun(f"""
SELECT ?facility ?tType ?tName WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?t schema:name ?tname }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?tname, REPLACE(STR(?t), '^.*/', '')) AS ?tName)
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?tType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?tType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?tType)
  }}
}}
LIMIT 5
""", limit=5)



### Universities con transporte cercano (conteos por tipo)
--- 3 filas ---
['Subway', '99']
['SuburbanTrain', '24']
['Bus', '2']
### Universities → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 5 filas ---
['Universidad Complutense de Madrid. Centro Complutense para la Enseñanza del Español', 'Subway', 'Metro: Ciudad Universitaria (línea 6)']
['Universidad Autónoma de Madrid. Facultad de Filosofía y Letras', 'SuburbanTrain', 'Cantoblanco Universidad C4']
['Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación', 'Subway', 'Metro: Sierra de Guadalupe (línea 1)']
['Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación', 'SuburbanTrain', 'Renfe: AtochaAlcalá de Henares (líneas C2 y C7) C1,C2,C3,C4,C5,C6,C7,C8 y C10']
['Residencia Universitaria Nebrija Arturo Soria', 'Subway', 'Metro: Arturo soria (línea 4)']

### PublicLibraries con transporte cercano (conteos por tipo)
--- 3 filas ---
['Subway', '45']
['SuburbanTrain', '13']
['Bus', '1

In [ ]:
# ✅ Número de individuos Facility

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:Facility   BIND("Facility"    AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")

--- 1 filas ---
['Facility', '309']


In [ ]:
# --------------------------------------------------------------------
query_2 = """
SELECT (COUNT(?park) AS ?total)  # Cambiamos ?count a ?total para evitar colisiones con el método
WHERE {
?park a schema:ParkGarden ;
        sc:accessibility "Instalación parcialmente accesible para personas con movilidad reducida" .
}
"""

print("\n--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---")
try:
    results_2 = g.query(PREFIXES + query_2)

    # COUNT devuelve una única fila. Iteramos sobre ella y accedemos al valor.
    for row in results_2:

        count_value = row.total.value if hasattr(row.total, 'value') else row.total
        print(f"Total de Parques Parcialmente Accesibles: {count_value}")


except Exception as e:
    print(f"Error al ejecutar la Consulta 2: {e}")


--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---
Total de Parques Parcialmente Accesibles: 197


In [ ]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:Facility","Facilities")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)


### Universities → Neighbourhood
--- 5 filas ---
['Escuela Técnica Superior de Ingenieros en Topografía, Geodesia y Cartografía', 'PUENTE DE VALLECAS']
['Parque Enrique Tierno Galván', 'ARGANZUELA']
['Parque Cerro Almodóvar', 'LATINA']
['Universidad Complutense de Madrid. Centro Superior de Idiomas Modernos', 'MONCLOA-ARAVACA']
['Centro Deportivo Municipal Francisco Fernández Ochoa', 'CARABANCHEL']


In [ ]:
# Numero de enlaces a wikidata de cada clase

qrun("""
SELECT ?class (COUNT(*) AS ?countWikidata)
WHERE {
  VALUES ?class {
    schema:Park
    schema:Library
    sc:SportsFacility
    sc:EducationalInstitution
    dbo:Municipality dbo:District dbo:Neighbourhood
  }
  ?s a ?class ; owl:sameAs ?o .
  FILTER( STRSTARTS(STR(?o), "http://www.wikidata.org/entity/")
       || STRSTARTS(STR(?o), "https://www.wikidata.org/entity/") )
}
GROUP BY ?class
ORDER BY ?class
""")

--- 7 filas ---
['http://dbpedia.org/ontology/District', '21']
['http://dbpedia.org/ontology/Municipality', '1']
['http://dbpedia.org/ontology/Neighbourhood', '113']
['http://smartcity.linkeddata.es/lcc/ontology#EducationalInstitution', '2']
['http://smartcity.linkeddata.es/lcc/ontology#SportsCenter', '2']
['https://schema.org/Library', '3']
['https://schema.org/ParkGarden', '4']


In [ ]:
# Ejemplos de esos enlaces

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters"),
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood")
]:

    print(f"\n### Ejemplo de {label} con acceso a wikidata")
    qrun(f"""
SELECT ?s ?o WHERE {{
  ?s a {subtype} ; owl:sameAs ?o .
  FILTER( STRSTARTS(STR(?o), "http://www.wikidata.org/entity/")
       || STRSTARTS(STR(?o), "https://www.wikidata.org/entity/") )
}}
LIMIT 10
""", limit=5)


### Ejemplo de Universities con acceso a wikidata
--- 2 filas ---
['http://smartcity.linkeddata.es/lcc/resource/facility/educationalInstitution/Universidad-Complutense-de-Madrid.-Facultad-de-Ciencias-Biologicas', 'http://www.wikidata.org/entity/Q47080407']
['http://smartcity.linkeddata.es/lcc/resource/facility/educationalInstitution/Escuela-Politecnica-de-Ensenanza-Superior-(EPES)', 'http://www.wikidata.org/entity/Q845392']

### Ejemplo de PublicLibraries con acceso a wikidata
--- 3 filas ---
['http://smartcity.linkeddata.es/lcc/resource/facility/library/Biblioteca-Publica-Elena-Fortun-(Retiro)', 'http://www.wikidata.org/entity/Q85697975']
['http://smartcity.linkeddata.es/lcc/resource/facility/library/Biblioteca-Publica-Hortaleza', 'http://www.wikidata.org/entity/Q85697866']
['http://smartcity.linkeddata.es/lcc/resource/facility/library/Biblioteca-Publica-Antonio-Mingote-(Latina)', 'http://www.wikidata.org/entity/Q85697873']

### Ejemplo de ParksGardens con acceso a wikidata
--- 4 fil

In [ ]:
# Verificar que no existen sameAs con no-URL

qrun("""
SELECT ?s ?o WHERE {
  ?s owl:sameAs ?o .
  FILTER( !( STRSTARTS(STR(?o), "http://") || STRSTARTS(STR(?o), "https://") ) )
}
LIMIT 20
""")

--- 0 filas ---
